In [1]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [2]:
pip install chromadb cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.1 MB/s eta 0:00:

In [5]:
pip install cohere

In [9]:
# Step 1: Extract Text from PDF and Split into Sentences
def extract_and_split_text_from_pdf(pdf_path):
    """
    Extracts the full text from a PDF and splits it into smaller sections.
    """
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            text += page_text
    # Split the text into sentences or small sections for better granularity
    sections = text.split('. ')
    return sections

In [8]:
# Initialize Cohere client
import cohere
COHERE_API_KEY = 'C1gKHRZE7Lp4vopHevghOMo5sdLCwDfOH31JFWNs'
co = cohere.Client(COHERE_API_KEY)

In [10]:
# Generate embeddings using Cohere
def generate_embeddings(text_list):
    response = co.embed(texts=text_list)
    embeddings = response.embeddings
    return embeddings

In [11]:
import chromadb
# Initialize Chroma client
client = chromadb.Client()

In [12]:
# Store the embeddings in Chroma
def store_embeddings_in_chroma(text_list, embeddings):
    """
    Store the extracted PDF text and their embeddings into Chroma database.
    If the collection already exists, use the existing collection.
    """
    collection_name = "pdf_embeddings_collection"

    # Check if the collection already exists
    try:
        collection = client.get_collection(collection_name)
        print(f"Using existing collection: {collection_name}")
    except:
        # If collection does not exist, create it
        collection = client.create_collection(name=collection_name)
        print(f"Created new collection: {collection_name}")

    # Add documents and embeddings to the collection
    for i, text in enumerate(text_list):
        collection.add(documents=[text], embeddings=[embeddings[i]], ids=[f'doc_{i}'])
    print("Embeddings stored in Chroma.")

In [13]:
# Search Chroma and refine results with LLM
def search_in_chroma(query):
    query_embedding = generate_embeddings([query])[0]

    # Perform search in Chroma
    collection = client.get_collection("pdf_embeddings_collection")
    results = collection.query(query_embeddings=[query_embedding], n_results=5)

    # Use LLM to refine the result
    relevant_text = results['documents'][0]

    # Refine using an LLM (like Cohere's generation model or any summarization model)
    response = co.generate(
        prompt=f"Question: {query}\nAnswer based on the following text: {relevant_text}",
        max_tokens=100  # Control the length of the response
    )

    return response.generations[0].text

In [14]:
from google.colab import files
uploaded = files.upload()


Saving Resume (13).pdf to Resume (13).pdf


In [15]:
file_name = list(uploaded.keys())[0]

# Store the path in the pdfPath variable
pdfPath = f"./{file_name}"


In [ ]:
import PyPDF2

# Main chatbot function
if __name__ == "__main__":
    pdf_path = pdfPath

    # Step 1: Extract and split the text
    print("Extracting and splitting text from the PDF...")
    text_sections = extract_and_split_text_from_pdf(pdf_path)

    # Step 2: Generate embeddings for each section
    print("Generating embeddings...")
    text_embeddings = generate_embeddings(text_sections)

    # Step 3: Store embeddings in Chroma
    print("Storing embeddings in Chroma...")
    store_embeddings_in_chroma(text_sections, text_embeddings)

    print("Chatbot is ready to answer questions.")

    # Chatbot loop
    while True:
        user_query = input("\nAsk a question based on the PDF content (or type 'exit' to quit): ")

        if user_query.lower() == 'exit':
            print("Exiting chatbot. Goodbye!")
            break

        # Search and refine the response
        response = search_in_chroma(user_query)
        print(f"\nAnswer: {response}\n")


Extracting and splitting text from the PDF...
Generating embeddings...
Storing embeddings in Chroma...
Created new collection: pdf_embeddings_collection
Embeddings stored in Chroma.
Chatbot is ready to answer questions.

Ask a question based on the PDF content (or type 'exit' to quit): cgpa



Answer:  The cumulative GPA of the individual listed in the provided resume is 7.81/10.0. 


Ask a question based on the PDF content (or type 'exit' to quit): is he know deep learning 



Answer:  According to the given resume, the individual has not explicitly mentioned any experience or certification in Deep Learning. However, it is evident that they have extensive experience in Full-Stack Software Development and numerous programming languages.

Would you like me to provide further insights based on the resume? 

